In [1]:
root = "C:/Users/egorg/Projects/MemcoinsTradingBotPolygon"; import sys; sys.path.append(root)

In [18]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import src.utils as util

from matplotlib.colors import TwoSlopeNorm
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from src.env.data import ParametersSet, ScalarFeatures, Dataset
from src.env.simulation import Market, Agent
from src.env.strategy import DipsStrategy

In [3]:
# pd.options.display.float_format = '{:,.8f}'.format

In [4]:
data_path = f"{root}/data"
shift = int(180 / 60)
datasets = util.load_datasets(data_path, shift, regimes_grid=pd.read_csv(f"{root}/data/dataset_regime_map.csv"))
len(datasets)

5049

In [5]:
features_selected: list[str] = ["rstd", "rsi_crossovers", "regression_slope", "regression_sqr"]

In [6]:
df = util.dataset_to_dataframe(datasets)
df = df[df["trial_rank"] == 0]
df

,dataset,std,rstd,ma_distance_ratio,rsi_crossovers,rsi_crossunders,entropy,regression_slope,regression_sqr,volatility_clustering,...,drawdown_convexity,drop_velocity,regime,trial_rank,profit,detection_period,drop_threshold,reversal_threshold,tp_pct,sl_pct
0,0_10,6.523627e-08,0.136456,0.048904,1,0,3.023363,0.002367,0.784891,0.045169,...,0.000027,0.711631,0,0,1005.423851,5,0.0460,0.03290,0.0491,0.00751
20,0_11,3.956624e-08,0.080538,0.027044,1,0,3.189496,-0.001863,0.944745,-0.086710,...,-0.000034,0.562611,1,0,1204.695376,5,0.1185,0.03895,0.0734,0.03944
40,0_12,2.014306e-08,0.044340,0.019405,1,0,2.179471,-0.000821,0.972760,-0.026485,...,0.000006,0.347130,2,0,1307.900000,3,0.0522,0.02336,0.0469,0.02719
60,0_13,2.400482e-08,0.056238,0.017800,1,0,2.219783,-0.001350,0.980654,-0.048117,...,-0.000014,0.271950,3,0,1365.978294,4,0.0509,0.02897,0.0327,0.02457
80,0_14,1.724124e-08,0.042506,0.024766,1,0,2.412590,-0.000219,0.966406,0.429391,...,-0.000012,2.363053,4,0,1144.787500,6,0.0912,0.07686,0.0964,0.08304
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100880,9_4,2.109971e-07,0.085310,0.021940,4,0,2.649813,-0.002241,0.866363,0.056508,...,-0.000018,0.431340,3,0,1314.253231,6,0.0425,0.01345,0.0374,0.03294
100900,9_5,1.526912e-07,0.067886,0.025938,4,0,1.965845,-0.001244,0.564301,0.122511,...,0.000015,0.268158,4,0,1040.690137,2,0.0491,0.01573,0.0721,0.00931
100920,9_6,2.925304e-07,0.120796,0.043184,2,1,1.646462,0.002239,0.470154,-0.016312,...,-0.000017,0.775788,5,0,1142.515861,4,0.0357,0.01171,0.0803,0.01613
100940,9_7,1.469328e-07,0.054975,0.034891,0,1,1.963904,0.000559,0.368129,-0.018669,...,-0.000016,0.204268,17,0,1207.897028,4,0.0310,0.01410,0.0689,0.04989


In [7]:
results = pd.read_csv(f"{data_path}/strategy_regime_map.csv")
results

,regime,value,detection_period,drop_threshold,reversal_threshold,tp_pct,sl_pct
0,0,1000.033583,3,0.1018,0.04004,0.0956,0.06546
1,0,1000.000000,1,0.1021,0.07403,0.0152,0.01202
2,0,1000.000000,4,0.1439,0.12657,0.0253,0.00383
3,0,1000.000000,1,0.1392,0.06336,0.0546,0.00696
4,0,1000.000000,2,0.1195,0.08565,0.0836,0.05336
...,...,...,...,...,...,...,...
835,41,1000.000000,5,0.1437,0.12891,0.0670,0.02680
836,41,1000.000000,3,0.1486,0.12208,0.0585,0.03005
837,41,1000.000000,3,0.1419,0.12127,0.0713,0.04233
838,41,1000.000000,1,0.1339,0.11287,0.0513,0.03263


In [8]:
rows = []
for i in range(42):
    rows.append(results[results["regime"] == i].iloc[0])
pd.DataFrame(rows)

,regime,value,detection_period,drop_threshold,reversal_threshold,tp_pct,sl_pct
0,0.0,1000.033583,3.0,0.1018,0.04004,0.0956,0.06546
20,1.0,1000.000000,3.0,0.1018,0.04004,0.0956,0.06546
40,2.0,1119.897278,3.0,0.1018,0.04004,0.0956,0.06546
60,3.0,1003.527079,2.0,0.1204,0.06092,0.0914,0.04784
80,4.0,1027.229955,6.0,0.1308,0.09024,0.0853,0.03693
100,5.0,1012.746740,1.0,0.0977,0.05991,0.0876,0.05626
120,6.0,1125.688690,6.0,0.0563,0.01769,0.1000,0.06940
140,7.0,1000.000000,3.0,0.1018,0.04004,0.0956,0.06546
160,8.0,1058.194209,2.0,0.1128,0.06564,0.0485,0.03625
180,9.0,1000.000000,1.0,0.1392,0.06336,0.0546,0.00696


In [43]:
def view_trades(regime: int, index: int, params: ParametersSet):
    regime_datasets = df[df["regime"] == regime]
    if index >= len(regime_datasets): return
    row = regime_datasets.iloc[index]
    market = Market(pd.read_csv(f"{root}/data/sliced/data_" + row["dataset"] + ".csv"))
    agent = Agent(capital=1000, commission=0.01)
    strategy = DipsStrategy(
        market, 
        agent,
        detection_period=params.detection_period,
        drop_threshold=params.drop_threshold,
        reversal_threshold=params.reversal_threshold,
        tp_pct=params.tp_pct,
        sl_pct=params.sl_pct
    )
    strategy.evaluate(verbose=True)

In [ ]:
#    0.0301             0.00983  0.0158 0.00278
for i in range(20):
    view_trades(35, i, ParametersSet(-1, 6, 0.0301, 0.00983, 0.0158, 0.00278))